In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install transformers
!pip install py-rouge

!git clone https://github.com/brs1977/transformers_summarization.git trans_bertsum

     |████████████████████████████████| 450kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 55.0MB/s 
     |████████████████████████████████| 870kB 46.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=b5755641aee56076f7db1acbd7ad43e33bcff1997904043aa4f298a84f8351b8
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 61kB 2.3MB/s 
Cloning into 'trans_bertsum'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 4), reused 17 (delta 3), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [3]:
#kaggle key
!mkdir ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle
!ls ~/.kaggle

kaggle.json


In [4]:
!kaggle competitions download -c title-generation

  0% 0.00/6.39M [00:00<?, ?B/s]
100% 6.39M/6.39M [00:00<00:00, 58.8MB/s]
  0% 0.00/905k [00:00<?, ?B/s]
100% 905k/905k [00:00<00:00, 129MB/s]
 80% 35.0M/43.7M [00:00<00:00, 67.2MB/s]
100% 43.7M/43.7M [00:00<00:00, 97.8MB/s]
  0% 0.00/778k [00:00<?, ?B/s]
100% 778k/778k [00:00<00:00, 107MB/s]


In [5]:
#%cd /content
!mkdir data
!unzip sample_submission.csv.zip -d data
!unzip vocs.pkl.zip -d data
!unzip train.csv.zip -d data
!mv test.csv data

Archive:  sample_submission.csv.zip
  inflating: data/sample_submission.csv  
Archive:  vocs.pkl.zip
  inflating: data/vocs.pkl           
Archive:  train.csv.zip
  inflating: data/train.csv          


In [6]:
%cd /content/trans_bertsum
import numpy as np

import random
import argparse
import logging
import os
import sys
from collections import namedtuple

import data_loader
from data_loader import load_dataset

import torch
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm

from modeling_bertabs import BertAbs, build_predictor
from transformers import BertTokenizer
from utils_dataset import (
    SummarizationDataset,
    build_mask,
    compute_token_type_ids,
    encode_for_summarization,
    fit_to_block_size,
)
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup)


/content/trans_bertsum


In [0]:
# os.path.isdir(path)
#args.batch_size = 32

In [7]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

def set_seed(args):
  random.seed(args.seed)
  np.random.seed(args.seed)
  torch.manual_seed(args.seed)

class ARGS(object):  
  model_name_or_path = ''         #"Path to pre-trained model or shortcut name"
  output_dir = ''                 # "The output directory where the model checkpoints and predictions will be written."
  config_name = ''                #"Pretrained config name or path if not the same as model_name"
  seed = 666
  bert_model = "bert-base-uncased"

  train_batch_size = 32
  num_train_epochs = 3            #"Total number of training epochs to perform."
  max_steps = -1                  #"If > 0: set total number of training steps to perform. Override num_train_epochs."
  logging_steps = 100             #"Log every X updates steps."
  save_steps = 3600               #"Save checkpoint every X updates steps."

  gradient_accumulation_steps = 8 #"Number of updates steps to accumulate before performing a backward/update pass."
  learning_rate = 5e-5            #"The initial learning rate for Adam."
  adam_epsilon = 1e-8             #"Epsilon for Adam optimizer."
  warmup_steps = 0.0              #"Linear warmup over warmup_steps."   
  max_grad_norm = 1.0             #"Max gradient norm."
  weight_decay = 0.0              #"Weight decay if we apply some."

  # n_best_size = 5                 #"The total number of n-best predictions to generate in the nbest_predictions.json output file."

  fp16 = False #"Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit"
  fp16_opt_level = 'O1' #"For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']." "See details at https://nvidia.github.io/apex/amp.html"

  bert_data_path = 'data/arxiv'
  max_pos = 512
  max_tgt_len = 140
  use_interval = True
  task = 'abs'


args = ARGS()

tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=True)
#tokenizer = BertTokenizer.from_pretrained(os.path.join(args.bert_model, 'vocab.txt'))

train_dataset = torch.load('/content/drive/My Drive/nlp/nlp_model/abs_bert/data130X30/train.pt')
model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
# model = BertAbs.from_pretrained(args.bert_model)

args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(args.device)

# Set seed
set_seed(args)



01/29/2020 05:24:23 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpqywxe6ve
01/29/2020 05:24:23 - INFO - transformers.file_utils -   copying /tmp/tmpqywxe6ve to cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
01/29/2020 05:24:23 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
01/29/2020 05:24:23 - INFO - transformers.file_utils -   removing temp file /tmp/tmpqywxe6ve
01/29/2020 05:24:23 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/t

In [36]:
!git pull origin master

From https://github.com/brs1977/transformers_summarization
 * branch            master     -> FETCH_HEAD
Already up to date.


In [54]:
# len(train_dataset[0])
# train_sampler = RandomSampler(train_dataset)
# train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, drop_last=True)
# for batch in train_dataloader:
#   break
# batch  


# data_loader.logger = logger
args.max_tgt_len = 96
train_dataloader = data_loader.Dataloader(args, load_dataset(args, 'train', shuffle=True), args.train_batch_size, args.device,
                                      shuffle=True, is_test=False)



01/29/2020 07:20:04 - INFO - __main__ -   Loading train dataset from data/arxiv.train.6.bert.pt, number of examples: 1419


In [0]:
Batch = namedtuple("Batch", ["document_names", "batch_size", "src", "segs", "mask_src", "tgt_str"])

def build_data_iterator(tokenizer):
    dataset = load_and_cache_examples(tokenizer)
    sampler = SequentialSampler(dataset)

    def collate_fn(data):
        return collate(data, tokenizer, block_size=512, device=args.device)

    iterator = DataLoader(dataset, sampler=sampler, batch_size=args.train_batch_size, collate_fn=collate_fn,)

    return iterator


def load_and_cache_examples(tokenizer):
    dataset = SummarizationDataset('/content/data/')
    return dataset


def collate(data, tokenizer, block_size, device):
    """ Collate formats the data passed to the data loader.

    In particular we tokenize the data batch after batch to avoid keeping them
    all in memory. We output the data as a namedtuple to fit the original BertAbs's
    API.
    """
    data = [x for x in data if not len(x[1]) == 0]  # remove empty_files
    names = [name for name, _, _ in data]
    summaries = [" ".join(summary_list) for _, _, summary_list in data]

    encoded_text = [encode_for_summarization(abstract, title, tokenizer) for _, abstract, title in data]
    encoded_stories = torch.tensor(
        [fit_to_block_size(abstract, block_size, tokenizer.pad_token_id) for abstract, _ in encoded_text]
    )
    encoder_token_type_ids = compute_token_type_ids(encoded_stories, tokenizer.cls_token_id)
    encoder_mask = build_mask(encoded_stories, tokenizer.pad_token_id)

    batch = Batch(
        document_names=names,
        batch_size=len(encoded_stories),
        src=encoded_stories, #.to(device),
        segs=encoder_token_type_ids, #.to(device),
        mask_src=encoder_mask, #.to(device),
        tgt_str=summaries,
    )

    return batch

data_iterator = build_data_iterator(tokenizer)

In [30]:

# !git pull origin master

import torch
for batch in data_iterator:
  print(len(batch))
  break

6


In [24]:
# !mkdir data
# !unzip /content/drive/My\ Drive/nlp/pre_sum_data.zip -d /content/trans_bertsum/data/

# dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])
#train_data  = torch.load('/content/trans_bertsum/data/arxiv.train.0.bert.pt')

train_data[0]['src_sent_labels']
# encoder_input_ids, decoder_input_ids, token_type_ids, encoder_attention_mask, decoder_attention_mask

#src, tgt, segs, clss, mask_src, mask_tgt, mask_cls

# train_data  = torch.load('/content/drive/My Drive/nlp/nlp_model/abs_bert/data130X30/train.pt')
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size, drop_last=True)


[1, 0, 0, 0, 0, 0, 0, 0]

In [0]:
def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    # train_sampler = RandomSampler(train_dataset)
    # train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
        os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 1
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=False
    )
    # Added here for reproductibility
    set_seed(args)

    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {
                "encoder_input_ids": batch[0],
                "decoder_input_ids": batch[1],
                "token_type_ids": batch[2],
                "encoder_attention_mask": batch[3],
                'decoder_attention_mask': batch[3]
            }

    # def forward(self, src, src_mask, tgt, tgt_mask):
    #     # src/tgt: [batch_size, seq_len]

    #     # shift right
    #     tgt = tgt[:, :-1]
    #     tgt_mask = tgt_mask[:, :-1]
    #     # bert input: BertModel.forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=True)
    #     # token_type_ids is not important since we only have one sentence so we can use default all zeros
    #     bert_encoded = self.bert_encoder(src, attention_mask=src_mask, output_all_encoded_layers=False)[0]  # [batch_size, seq_len, hidden_size]
    #     # transformer input: BertDecoder.forward(self, tgt_seq_embedded, tgt_pos, src_seq, enc_output, return_attns=False)
    #     logits = self.decoder(tgt, src, bert_encoded)  # [batch_size, seq_len, vocab_size]


            outputs = model(**inputs)
            # model outputs are always tuple in transformers (see doc)
            loss = outputs[0]

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                # Log metrics
                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    
                    # Only evaluate when single GPU otherwise metrics may not average well
                    # if args.evaluate_during_training:
                    #     results = evaluate(args, model, tokenizer)
                    #     for key, value in results.items():
                    #         tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                # Save model checkpoint
                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    # Take care of distributed/parallel training
                    model_to_save = model.module if hasattr(model, "module") else model
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    tb_writer.close()

    return global_step, tr_loss / global_step


In [38]:
train(args, data_iterator, model, tokenizer)

01/26/2020 10:42:49 - INFO - __main__ -   ***** Running training *****
01/26/2020 10:42:49 - INFO - __main__ -     Num examples = 121489
01/26/2020 10:42:49 - INFO - __main__ -     Num Epochs = 3
01/26/2020 10:42:49 - INFO - __main__ -     Gradient Accumulation steps = 8
01/26/2020 10:42:49 - INFO - __main__ -     Total optimization steps = 1581









Epoch:   0%|          | 0/3 [00:00<?, ?it/s]









Iteration:   0%|          | 0/3 [00:00<?, ?it/s]











TypeError: ignored

In [0]:
# batch.keys()
# dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])
# data_loader.Dataloader(args, load_dataset(args, 'train', shuffle=True), args.batch_size, device,
#                                       shuffle=True, is_test=False)

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size, drop_last=True)


In [87]:
# for _ in train_iterator:
#       epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)

args.max_tgt_len = 96
train_dataloader = data_loader.Dataloader(args, load_dataset(args, 'train', shuffle=True), 512, args.device,
                                      shuffle=True, is_test=False)

for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration", disable=False)):
  # print(step,batch)

  # batch = tuple(t.to(args.device) for t in batch)
# setattr(self, 'src', src.to(device))
#             setattr(self, 'tgt', tgt.to(device))
#             setattr(self, 'segs', segs.to(device))
#             setattr(self, 'mask_src', mask_src.to(device))
#             setattr(self, 'mask_tgt', mask_tgt.to(device))

  inputs = {
      "encoder_input_ids": batch.src,
      "decoder_input_ids": batch.tgt,
      "token_type_ids": batch.segs,
      "encoder_attention_mask": batch.mask_src,
      'decoder_attention_mask': batch.mask_tgt
  }

  outputs = model(**inputs)
  # model outputs are always tuple in transformers (see doc)
  loss = outputs[0]
  print(loss)

  break
 

01/29/2020 07:56:59 - INFO - __main__ -   Loading train dataset from data/arxiv.train.3.bert.pt, number of examples: 18185
Iteration: 0it [00:00, ?it/s]


RuntimeError: ignored

In [88]:
133632 / 8 / 96
# 112 * 8 * 96
# 78336 / (8 * 96)
# shape '[138, -1, 8, 96]' is invalid for input of size 18432
# [163, -1, 8, 96]

174.0

In [0]:
Batch = namedtuple("Batch", ["document_names", "batch_size", "src", "segs", "mask_src", "tgt_str"])
# encoder_input_ids, decoder_input_ids, token_type_ids, encoder_attention_mask, decoder_attention_mask

def build_data_iterator(tokenizer):
    dataset = load_and_cache_examples(tokenizer)
    sampler = SequentialSampler(dataset)

    def collate_fn(data):
        return collate(data, tokenizer, block_size=512, device=args.device)

    iterator = DataLoader(dataset, sampler=sampler, batch_size=args.train_batch_size, collate_fn=collate_fn,)

    return iterator


def load_and_cache_examples(tokenizer):
    dataset = SummarizationDataset('/content/data/')
    return dataset


def collate(data, tokenizer, block_size, device):
    """ Collate formats the data passed to the data loader.

    In particular we tokenize the data batch after batch to avoid keeping them
    all in memory. We output the data as a namedtuple to fit the original BertAbs's
    API.
    """
    data = [x for x in data if not len(x[1]) == 0]  # remove empty_files
    names = [name for name, _, _ in data]
    summaries = [" ".join(summary_list) for _, _, summary_list in data]

    encoded_text = [encode_for_summarization(abstract, title, tokenizer) for _, abstract, title in data]
    encoded_stories = torch.tensor(
        [fit_to_block_size(abstract, block_size, tokenizer.pad_token_id) for abstract, _ in encoded_text]
    )
    encoder_token_type_ids = compute_token_type_ids(encoded_stories, tokenizer.cls_token_id)
    encoder_mask = build_mask(encoded_stories, tokenizer.pad_token_id)

    batch = Batch(
        document_names=names,
        batch_size=len(encoded_stories),
        src=encoded_stories, #.to(device),
        segs=encoder_token_type_ids, #.to(device),
        mask_src=encoder_mask, #.to(device),
        tgt_str=summaries,
    )

    return batch

data_iterator = build_data_iterator(tokenizer)

In [0]:
for batch in data_iterator:
  print(batch[2][0])
  break